# GIFT vs Non-GIFT Extended Validation Test

**Purpose**: Comprehensive validation of GIFT selectivity with real Dirichlet L-function zeros.

**Previous result**: GIFT conductors showed 1.7× better Fibonacci constraint than non-GIFT (0.73 vs 1.24).

**This test**: Extended conductor set for stronger statistical power.

---

In [ ]:
import mpmath as mp
import numpy as np
from scipy import stats
import json
import time

mp.mp.dps = 25
print(f"mpmath ready, precision: {mp.mp.dps} digits")

In [ ]:
# Extended conductor classification

# GIFT conductors (primes that ARE GIFT constants)
GIFT_PRIMES = {
    3: 'N_gen (generations)',
    5: 'Weyl / F₅',
    7: 'dim(K₇)',
    11: 'D_bulk',
    13: 'F₇',
}

# Non-GIFT primes (no simple GIFT interpretation)
NON_GIFT_PRIMES = {
    19: 'Prime (no GIFT)',
    23: 'Prime (no GIFT)',
    29: 'Prime (no GIFT)',
    31: 'Prime (no GIFT)',
    37: 'Prime (no GIFT)',
    41: 'Prime (no GIFT)',
    43: 'Prime (no GIFT)',
}

# Borderline / ambiguous (for separate analysis)
BORDERLINE = {
    17: 'dim(G₂) + N_gen = 14 + 3 (composite GIFT)',
    2: 'p₂ (Pontryagin)',
}

print(f"GIFT primes: {list(GIFT_PRIMES.keys())}")
print(f"Non-GIFT primes: {list(NON_GIFT_PRIMES.keys())}")
print(f"Borderline: {list(BORDERLINE.keys())}")

In [ ]:
# L-function computation (from previous notebook)

def legendre_symbol(a, p):
    if a % p == 0:
        return 0
    result = pow(a % p, (p - 1) // 2, p)
    return 1 if result == 1 else -1

def dirichlet_L_function(s, q, terms=8000):
    result = mp.mpc(0)
    for n in range(1, terms + 1):
        chi_n = legendre_symbol(n, q) if q > 2 else (1 if n % 2 == 1 else 0)
        if chi_n != 0:
            result += mp.mpc(chi_n) / mp.power(n, s)
    return result

def find_zeros_dirichlet(q, num_zeros=50, T_max=100, step=0.25, terms=8000):
    zeros = []
    t = 1.0
    prev_val = dirichlet_L_function(mp.mpc(0.5, t), q, terms)
    prev_sign = 1 if prev_val.real > 0 else -1
    
    while len(zeros) < num_zeros and t < T_max:
        t += step
        curr_val = dirichlet_L_function(mp.mpc(0.5, t), q, terms)
        curr_sign = 1 if curr_val.real > 0 else -1
        
        if curr_sign != prev_sign and prev_sign != 0:
            t_low, t_high = t - step, t
            for _ in range(15):
                t_mid = (t_low + t_high) / 2
                val_mid = dirichlet_L_function(mp.mpc(0.5, t_mid), q, terms)
                sign_mid = 1 if val_mid.real > 0 else -1
                val_low = dirichlet_L_function(mp.mpc(0.5, t_low), q, terms)
                sign_low = 1 if val_low.real > 0 else -1
                if sign_low != sign_mid:
                    t_high = t_mid
                else:
                    t_low = t_mid
            zero_t = (t_low + t_high) / 2
            if not zeros or abs(zero_t - zeros[-1]) > 0.1:
                zeros.append(zero_t)
        
        prev_val = curr_val
        prev_sign = curr_sign
    
    return zeros

print("L-function computation ready.")

In [ ]:
# Recurrence fitting
GIFT_LAGS = [5, 8, 13, 27]

def fit_recurrence(zeros, lags=GIFT_LAGS):
    max_lag = max(lags)
    n_points = len(zeros) - max_lag
    if n_points < 5:
        return None, float('inf')
    
    X = np.zeros((n_points, len(lags) + 1))
    y = np.zeros(n_points)
    
    for i in range(n_points):
        idx = i + max_lag
        y[i] = zeros[idx]
        for j, lag in enumerate(lags):
            X[i, j] = zeros[idx - lag]
        X[i, -1] = 1
    
    try:
        coeffs, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
        y_pred = X @ coeffs
        error = np.sqrt(np.mean((y - y_pred)**2))
        return coeffs, error
    except:
        return None, float('inf')

def compute_R(coeffs, lags=GIFT_LAGS):
    if coeffs is None:
        return None
    a8 = coeffs[lags.index(8)]
    a13 = coeffs[lags.index(13)]
    if abs(a13) < 1e-10:
        return None
    return (8 * a8) / (13 * a13)

print("Recurrence fitting ready.")

---
## Main Computation

⏱️ This will take ~30-40 minutes for all conductors

In [ ]:
# Combine all conductors
all_conductors = {}
for q, desc in GIFT_PRIMES.items():
    all_conductors[q] = ('gift', desc)
for q, desc in NON_GIFT_PRIMES.items():
    all_conductors[q] = ('non_gift', desc)
for q, desc in BORDERLINE.items():
    all_conductors[q] = ('borderline', desc)

# Sort by conductor
sorted_conductors = sorted(all_conductors.keys())

print(f"Total conductors to test: {len(sorted_conductors)}")
print(f"Conductors: {sorted_conductors}")
print(f"\nEstimated time: ~{len(sorted_conductors) * 3} minutes")

In [ ]:
# Compute zeros and fit recurrence for all conductors
NUM_ZEROS = 50
T_MAX = 90
STEP = 0.25
TERMS = 8000

results = []
zeros_cache = {}

print(f"Computing {NUM_ZEROS} zeros per conductor...\n")
print("=" * 70)

total_start = time.time()

for i, q in enumerate(sorted_conductors):
    category, desc = all_conductors[q]
    
    print(f"[{i+1}/{len(sorted_conductors)}] q={q} ({category})...", end=" ", flush=True)
    start = time.time()
    
    # Skip q=2 (special case)
    if q == 2:
        print("SKIPPED (q=2 special case)")
        continue
    
    zeros = find_zeros_dirichlet(q, NUM_ZEROS, T_MAX, STEP, TERMS)
    elapsed = time.time() - start
    
    if len(zeros) >= 35:
        zeros_cache[q] = zeros
        coeffs, error = fit_recurrence(zeros, GIFT_LAGS)
        R = compute_R(coeffs, GIFT_LAGS)
        
        if R is not None:
            R_dev = abs(R - 1)
            results.append({
                'q': q,
                'category': category,
                'description': desc,
                'R': float(R),
                'R_deviation': float(R_dev),
                'num_zeros': len(zeros),
            })
            print(f"{len(zeros)} zeros, R={R:.3f}, |R-1|={R_dev:.3f} ({elapsed:.0f}s)")
        else:
            print(f"{len(zeros)} zeros, R=N/A ({elapsed:.0f}s)")
    else:
        print(f"only {len(zeros)} zeros - SKIPPED ({elapsed:.0f}s)")

total_elapsed = time.time() - total_start
print("=" * 70)
print(f"\n✓ Completed in {total_elapsed/60:.1f} minutes")
print(f"✓ {len(results)} conductors with valid results")

---
## Statistical Analysis

In [ ]:
# Group results
gift_results = [r for r in results if r['category'] == 'gift']
non_gift_results = [r for r in results if r['category'] == 'non_gift']
borderline_results = [r for r in results if r['category'] == 'borderline']

def analyze_group(group, name):
    if not group:
        return None, None, None
    devs = [r['R_deviation'] for r in group]
    Rs = [r['R'] for r in group]
    mean_dev = np.mean(devs)
    std_dev = np.std(devs)
    median_dev = np.median(devs)
    
    print(f"\n{name}:")
    print(f"  n = {len(group)}")
    print(f"  Conductors: {[r['q'] for r in group]}")
    print(f"  Mean |R-1|: {mean_dev:.4f} ± {std_dev:.4f}")
    print(f"  Median |R-1|: {median_dev:.4f}")
    print(f"  R values: {[f'{r:.2f}' for r in Rs]}")
    print(f"  Best: q={group[np.argmin(devs)]['q']} (|R-1|={min(devs):.3f})")
    print(f"  Worst: q={group[np.argmax(devs)]['q']} (|R-1|={max(devs):.3f})")
    
    return mean_dev, std_dev, median_dev

print("=" * 70)
print("GROUP STATISTICS")
print("=" * 70)

gift_mean, gift_std, gift_median = analyze_group(gift_results, "GIFT PRIMES")
non_gift_mean, non_gift_std, non_gift_median = analyze_group(non_gift_results, "NON-GIFT PRIMES")
border_mean, border_std, border_median = analyze_group(borderline_results, "BORDERLINE")

In [ ]:
# Statistical tests
print("\n" + "=" * 70)
print("HYPOTHESIS TESTS")
print("=" * 70)

if gift_results and non_gift_results:
    gift_devs = [r['R_deviation'] for r in gift_results]
    non_gift_devs = [r['R_deviation'] for r in non_gift_results]
    
    # t-test
    t_stat, p_two = stats.ttest_ind(gift_devs, non_gift_devs)
    
    # Mann-Whitney U (non-parametric)
    u_stat, p_mann = stats.mannwhitneyu(gift_devs, non_gift_devs, alternative='less')
    
    print(f"\nH₀: GIFT and Non-GIFT have same |R-1|")
    print(f"H₁: GIFT has LOWER |R-1| (better Fibonacci constraint)")
    print(f"\nGIFT mean:     {gift_mean:.4f}")
    print(f"Non-GIFT mean: {non_gift_mean:.4f}")
    print(f"Ratio:         {non_gift_mean/gift_mean:.2f}× (GIFT is better)" if gift_mean < non_gift_mean else f"Ratio: {gift_mean/non_gift_mean:.2f}× (Non-GIFT is better)")
    print(f"\nt-test:")
    print(f"  t-statistic: {t_stat:.3f}")
    print(f"  p-value (two-tailed): {p_two:.4f}")
    print(f"  p-value (one-tailed): {p_two/2:.4f}")
    print(f"\nMann-Whitney U:")
    print(f"  U-statistic: {u_stat:.1f}")
    print(f"  p-value (one-tailed): {p_mann:.4f}")
    
    if gift_mean < non_gift_mean:
        print(f"\n→ GIFT outperforms Non-GIFT")
        if p_two/2 < 0.05:
            print(f"→ SIGNIFICANT at p < 0.05 ✓✓")
        elif p_two/2 < 0.10:
            print(f"→ SIGNIFICANT at p < 0.10 ✓")
        else:
            print(f"→ Not significant (p = {p_two/2:.3f})")
    else:
        print(f"\n→ Non-GIFT outperforms GIFT (unexpected!)")

In [ ]:
# Ranking
print("\n" + "=" * 70)
print("COMPLETE RANKING (lower |R-1| = better)")
print("=" * 70)

sorted_results = sorted(results, key=lambda x: x['R_deviation'])

print(f"\n{'Rank':<5} {'q':<4} {'Category':<12} {'R':>8} {'|R-1|':>8} {'Description'}")
print("-" * 70)

for i, r in enumerate(sorted_results, 1):
    marker = "★" if r['category'] == 'gift' else "·" if r['category'] == 'non_gift' else "◇"
    print(f"{i:<5} {r['q']:<4} {r['category']:<12} {r['R']:>8.3f} {r['R_deviation']:>8.4f} {r['description'][:30]} {marker}")

print("\nLegend: ★ = GIFT, · = Non-GIFT, ◇ = Borderline")

In [ ]:
# Final verdict
print("\n" + "#" * 70)
print("#" + " FINAL VERDICT ".center(68) + "#")
print("#" * 70)

gift_is_better = gift_mean < non_gift_mean if (gift_mean and non_gift_mean) else False
ratio = non_gift_mean / gift_mean if gift_is_better else (gift_mean / non_gift_mean if gift_mean and non_gift_mean else 0)

print(f"\nGIFT conductors:     mean |R-1| = {gift_mean:.4f}" if gift_mean else "GIFT: N/A")
print(f"Non-GIFT conductors: mean |R-1| = {non_gift_mean:.4f}" if non_gift_mean else "Non-GIFT: N/A")

if gift_is_better:
    print(f"\n✓ GIFT shows {ratio:.1f}× BETTER Fibonacci constraint")
    print(f"\n→ The GIFT hypothesis is SUPPORTED by real L-function data!")
    print(f"→ Conductors related to K₇ topology show preferential structure")
    print(f"→ in Dirichlet L-function zeros.")
else:
    print(f"\n✗ GIFT does NOT show better Fibonacci constraint")
    print(f"→ The GIFT hypothesis is NOT supported.")

# Best performer
best = sorted_results[0]
print(f"\nBest performer: q = {best['q']} ({best['description']})")
print(f"                |R-1| = {best['R_deviation']:.4f}")

print("\n" + "#" * 70)

In [ ]:
# Save results
output = {
    'test': 'GIFT vs Non-GIFT Extended Validation',
    'method': 'Real Dirichlet L-function zeros (mpmath)',
    'parameters': {
        'num_zeros': NUM_ZEROS,
        'T_max': T_MAX,
        'series_terms': TERMS
    },
    'results': results,
    'summary': {
        'gift_conductors': [r['q'] for r in gift_results],
        'non_gift_conductors': [r['q'] for r in non_gift_results],
        'gift_mean': float(gift_mean) if gift_mean else None,
        'gift_std': float(gift_std) if gift_std else None,
        'non_gift_mean': float(non_gift_mean) if non_gift_mean else None,
        'non_gift_std': float(non_gift_std) if non_gift_std else None,
        'ratio': float(ratio) if ratio else None,
        'gift_is_better': bool(gift_is_better),
        'p_value_ttest': float(p_two/2) if 'p_two' in dir() else None,
        'p_value_mannwhitney': float(p_mann) if 'p_mann' in dir() else None,
    }
}

with open('gift_validation_extended.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Results saved to 'gift_validation_extended.json'")
print("\nPlease share this JSON file!")

---

## Summary

This extended test compares:
- **GIFT primes**: {3, 5, 7, 11, 13} — constants from K₇/E₈/G₂ structure
- **Non-GIFT primes**: {19, 23, 29, 31, 37, 41, 43} — no GIFT interpretation

If GIFT shows significantly lower |R-1|, it supports the hypothesis that K₇ topology is encoded in L-function zeros.

---
*GIFT Framework — February 2026*